## Averaging attention distributions from the last four layers then classifying via a Linear Network

We extract the attention distribution directly from a model that has been pretrained on bias detection!

We then determine the predicted biased word, and use only the attention distribution associated with that extracted word for further classification. 

We average together the last four attention distributions of the input. The classification model is a simple super shallow neural network.

In [1]:
import sys; sys.path.append("../../../../..")
import torch 
from src.experiment import AttentionExperiment, ClassificationExperiment
from src.dataset import ExperimentDataset
from src.params import Params
from src.utils.attention_utils import reduce_attention_dist, return_idx_attention_dist
from src.utils.classification_utils import run_bootstrapping
from src.utils.shared_utils import get_bias_predictions

In [2]:
%load_ext autoreload
%autoreload 2

In [27]:
params = Params.read_params("linear-params.json")
print("model = {}".format(params.final_task['model']))
print("layers = {}".format(params.intermediary_task["attention"]["layers"]))
print("reducer = {}".format(params.intermediary_task["attention"]["reducer"]))

model = shallow_nn
layers = [8, 9, 10, 11]
reducer = avg


In [4]:
# Loading in the dataset that we are using in this experiments 
# typically this dataset is the small set of ground-truth labels
dataset = ExperimentDataset.init_dataset(params.dataset)

03/21/2020 16:23:17 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at ./cache/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
386it [00:00, 4280.82it/s]


### Attention Experiment: 
* Is a class that wraps useful methods to extract attention distributions from a given BERT-based model 
* The user has to provide in two config files: One to specify parameters for how the attention scores should be extracted and combined, and other to specify the intermediary model from which the attention scores should be extracted from
* The user needs to instantiate the attention experiment with a function that tells the model how to run 
 inference on the given model. The function header is specified below: 
 
 ``` def initialize_attention_experiment(cls, intermediary_task_params, dataset_params, verbose=False) ```
 


In [5]:
attention_dataloader = dataset.return_dataloader() 
attention_experiment = AttentionExperiment.initialize_attention_experiment(params.intermediary_task, params.dataset, verbose=True)

03/21/2020 16:23:20 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at ./cache/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
03/21/2020 16:23:20 - INFO - pytorch_pretrained_bert.modeling -   loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at ./cache/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
03/21/2020 16:23:20 - INFO - pytorch_pretrained_bert.modeling -   extracting archive file ./cache/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir /tmp/tmpwyyz2lkp
03/21/2020 16:23:25 - INFO - pytorch_pretrained_bert.modeling -   Model config {
  "attention_probs_d

Instantiated joint model with pretrained weights.
Succesfully loaded in attention experiment!


```extract_attention_scores()``` works out of the box because the attention experiment has the config file saved, and knows what BERT model to use/load in, which layers to extract the attention scores from, and what the inference function is that should be used on this particular BERT model.

Attention_scores is then a list of dictionaries. The keys in this dictionary are the specific layers of a BERT model and the values are the corresponding attention distributions extracted from that particular layer.

In [6]:
attention_scores = attention_experiment.extract_attention_scores(attention_dataloader)

Getting the predictions from the BERT model trained to detect bias, and using those to index into the attention scores

In [7]:
bias_predictions = get_bias_predictions(dataset, params.intermediary_task, params.dataset, batch_size=2)

03/21/2020 16:24:03 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at ./cache/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
03/21/2020 16:24:03 - INFO - pytorch_pretrained_bert.modeling -   loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at ./cache/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
03/21/2020 16:24:03 - INFO - pytorch_pretrained_bert.modeling -   extracting archive file ./cache/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir /tmp/tmppd8ukja2
03/21/2020 16:24:07 - INFO - pytorch_pretrained_bert.modeling -   Model config {
  "attention_probs_d

In [8]:
bias_indices = torch.argmax(bias_predictions == 1, dim=1).tolist()

In [9]:
attention_scores_indexed = return_idx_attention_dist(attention_scores, bias_indices)
reduced_attention = reduce_attention_dist(attention_scores_indexed, params.intermediary_task["attention"]["reducer"])
stacked_reduced_attention = torch.stack(reduced_attention).squeeze()
dataset.add_data(stacked_reduced_attention, "attention_dist")
dataset.shuffle_data()
print("reduced_attention[0].shape = {}".format(reduced_attention[0].shape))
print("stacked_reduced_attention.shape = {}".format(stacked_reduced_attention.shape))
dataset

reduced_attention[0].shape = torch.Size([80])
stacked_reduced_attention.shape = torch.Size([324, 80])


Length: 324 Keys: dict_keys(['pre_ids', 'masks', 'pre_lens', 'post_in_ids', 'post_out_ids', 'pre_tok_label_ids', 'post_tok_label_ids', 'rel_ids', 'pos_ids', 'categories', 'index', 'bias_label', 'attention_dist'])

### This is where the classification experiment starts

We create a classification experiment that contains useful methods for classifying bias based on the attention distributions. 

In [60]:
params = Params.read_params("linear-params.json")


In [61]:
classification_experiment = ClassificationExperiment.init_cls_experiment(params.final_task)

In [ ]:
stats = run_bootstrapping(classification_experiment, dataset, params.final_task, num_bootstrap_iters=5, input_key="attention_dist", label_key="bias_label")

120 Epochs - 15 hidden

In [51]:
stats

{'auc': [(0.5738069872835498, 0.7272416935153508), 0.6466578022060107],
 'accuracy': [(0.5375, 0.7375), 0.621875]}

120 Epochs - 10 hidden

In [55]:
stats

{'auc': [(0.5459615349129416, 0.6317805590790788), 0.5891625632723232],
 'accuracy': [(0.56875, 0.6859375), 0.634375]}

120 Epochs - 20 hidden

In [59]:
stats

{'auc': [(0.5796915584415584, 0.6421708992761624), 0.615168371188495],
 'accuracy': [(0.5781250000000001, 0.7), 0.634375]}

130 Epochs - 15 hidden

In [ ]:
stats

Changing Epoch Count 

In [47]:
stats #200

{'auc': [(0.5391350744843393, 0.634156314699793), 0.5891580565472576],
 'accuracy': [(0.5781250000000001, 0.7140625), 0.634375]}

In [42]:
stats #170

{'auc': [(0.5270642248376624, 0.6466106757468314), 0.5775699765250323],
 'accuracy': [(0.5140625, 0.684375), 0.628125]}

In [38]:
stats # 120 

{'auc': [(0.446763970080786, 0.6524386724386725), 0.5928089859132059],
 'accuracy': [(0.5546875, 0.6828124999999999), 0.625]}

In [34]:
stats # 150

{'auc': [(0.4934778741962372, 0.6126634552042161), 0.5406702799516856],
 'accuracy': [(0.5859375000000001, 0.7015625), 0.659375]}

In [31]:
stats # 100 

{'auc': [(0.4258260704644176, 0.6054380784999708), 0.5371805776236259],
 'accuracy': [(0.546875, 0.728125), 0.621875]}

In [13]:
classification_experiment.save_model_weights("linear-attention.weights")